In [2]:
import os
os.chdir("../")

In [3]:
def map_to_stock_variable_name(name, prefix="bert"):
    name = name.split(":")[0]
    ns   = name.split("/")
    pns  = prefix.split("/")
#     print("rand" , name, ns, pns)
    if ns[:len(pns)] != pns:
        return None

    name = "/".join(["bert"] + ns[len(pns):])
    ns   = name.split("/")

    if ns[1] not in ["encoder", "embeddings"]:
        return None
    if ns[1] == "embeddings":
        if ns[2] == "LayerNorm":
            return name
        else:
            return "/".join(ns[:-1])
    if ns[1] == "encoder":
        if ns[3] == "intermediate":
            return "/".join(ns[:4] + ["dense"] + ns[4:])
        else:
            return name
    return None

def map_to_stock_variable_name_v2(name, prefix="bert"):
    name = name.split(":")[0].replace("embeddings_1", "embeddings")
    return name

def load_stock_weights(bert, ckpt_file):
    
    ckpt_reader = tf.train.load_checkpoint(ckpt_file)

    bert_prefix = bert.weights[0].name.split("/")[0]
    print(bert_prefix)
    weights = []
    for idx, weight in enumerate(bert.weights):
        print(idx, weight.name)
        stock_name = map_to_stock_variable_name_v2(weight.name, bert_prefix)
        
        if ckpt_reader.has_tensor(stock_name):
            
            value = ckpt_reader.get_tensor(stock_name)
#             print("stock_name" , stock_name, "value", value)
            weights.append(value)
            
        else:
#             print("stock_name" , stock_name)
            print("loader: No value for:[{}], i.e.:[{}] in:[{}]".format(weight.name, stock_name, ckpt_file))
            # raise ValueError("No value for:[{}], i.e.:[{}] in:[{}]".format(weight.name, stock_name, ckpt_file))
            weights.append(weight.value())

    bert.set_weights(weights)
    print("Done loading {} BERT weights from: {} into {} (prefix:{})".format(
        len(weights), ckpt_file, bert, bert_prefix))

In [4]:
ls /Users/naval/Desktop/Naval/CBM/BERT-NER-master/bert/multi_cased_L-12_H-768_A-12

bert_config.json                     bert_model.ckpt.meta
bert_model.ckpt.data-00000-of-00001  vocab.txt
bert_model.ckpt.index


In [5]:
import tokenization

class InputFeatures(object):
	"""A single set of features of data."""

	def __init__(self, input_ids, input_mask, segment_ids, label_ids,):
		self.input_ids = input_ids
		self.input_mask = input_mask
		self.segment_ids = segment_ids
		self.label_ids = label_ids
		#self.label_mask = label_mask

#### Load labels mapping
tokenizer_file_path = "/Users/naval/Desktop/Naval/CBM/BERT-NER-master/bert/multi_cased_L-12_H-768_A-12/vocab.txt"
tokenizer = tokenization.FullTokenizer(vocab_file=tokenizer_file_path, do_lower_case=False)
        
def convert_single_example(example):
    textlist = example.split(' ')
    tokens = []
    label_ids = [0]
    for i, word in enumerate(textlist):
        token = tokenizer.tokenize(word)
        tokens.extend(token)
    # tokens = tokenizer.tokenize(example.text)
    if len(tokens) >= max_seq_length - 1:
        tokens = tokens[0:(max_seq_length - 2)]
    ntokens = []
    segment_ids = []
    ntokens.append("[CLS]")
    segment_ids.append(0)
    # append("O") or append("[CLS]") not sure!
    label_ids.append(label_map["[CLS]"])
    for i, token in enumerate(tokens):
        ntokens.append(token)
        segment_ids.append(0)
    ntokens.append("[SEP]")
    segment_ids.append(0)
    # append("O") or append("[SEP]") not sure!
    input_ids = tokenizer.convert_tokens_to_ids(ntokens)
    input_mask = [1] * len(input_ids)
    #label_mask = [1] * len(input_ids)
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)
        # we don't concerned about it!
        ntokens.append("**NULL**")
        #label_mask.append(0)
    # print(len(input_ids))
    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    #assert len(label_mask) == max_seq_length

    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=label_ids,
        #label_mask = label_mask
    )
    return feature

In [6]:
max_seq_length = 128

In [7]:
label_list = ["," , ".", "?", "O" , "X","[CLS]","[SEP]"]
label_map = {}
for (i, label) in enumerate(label_list,1):
    label_map[label] = i

In [8]:
convert = convert_single_example("xx ans naval")

In [9]:
import tensorflow as tf
import modeling as modeling
from tensorflow import keras

max_seq_len = 128

# allow gpu growth
# tf.config.gpu.set_per_process_memory_growth(True)

# bert config
init_checkpoint = "/Users/naval/Desktop/Naval/CBM/BERT-NER-master/bert/multi_cased_L-12_H-768_A-12/bert_model.ckpt"
bert_config_file = "/Users/naval/Desktop/Naval/CBM/BERT-NER-master/bert/multi_cased_L-12_H-768_A-12/bert_config.json"
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

# fake input for initializing weights
# input_ids = tf.Variable(tf.ones(shape=(1, 512), dtype=tf.int32))
# input_ids      = tf.keras.layers.Input(shape=(max_seq_len), dtype='int32', name="input_ids")
input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_ids")
token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="token_type_ids")
input_mask = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_mask")
bert = modeling.BertModel(bert_config, is_training=True)


# print("Loading weights")
# bert.load_weights(init_checkpoint)
# print("Weights loadad!!!")

In [10]:
output = bert(input_ids, input_mask=input_mask, token_type_ids=token_type_ids)

In [11]:
output

<tf.Tensor 'bert/Identity:0' shape=(None, 768) dtype=float32>

In [12]:
output = tf.keras.layers.Dense(2)(output)

In [13]:

model = tf.keras.Model(inputs=[input_ids, input_mask, token_type_ids], outputs=output)
model.build(input_shape=[(None, max_seq_len),
                                 (None, max_seq_len), (None, max_seq_len)])

In [14]:
output

<tf.Tensor 'dense/Identity:0' shape=(None, 2) dtype=float32>

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (BertModel)                (None, 768)          177853440   input_ids[0][0]                  
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
______________________________________________________________________________________________

In [16]:
load_stock_weights(model, init_checkpoint)

bert
0 bert/embeddings/word_embeddings:0
1 bert/embeddings_1/token_type_embeddings:0
2 bert/embeddings_1/position_embeddings:0
3 bert/embeddings_1/LayerNorm/gamma:0
4 bert/embeddings_1/LayerNorm/beta:0
5 bert/encoder/layer_0/attention/self/query/kernel:0
6 bert/encoder/layer_0/attention/self/query/bias:0
7 bert/encoder/layer_0/attention/self/key/kernel:0
8 bert/encoder/layer_0/attention/self/key/bias:0
9 bert/encoder/layer_0/attention/self/value/kernel:0
10 bert/encoder/layer_0/attention/self/value/bias:0
11 bert/encoder/layer_1/attention/self/query/kernel:0
12 bert/encoder/layer_1/attention/self/query/bias:0
13 bert/encoder/layer_1/attention/self/key/kernel:0
14 bert/encoder/layer_1/attention/self/key/bias:0
15 bert/encoder/layer_1/attention/self/value/kernel:0
16 bert/encoder/layer_1/attention/self/value/bias:0
17 bert/encoder/layer_2/attention/self/query/kernel:0
18 bert/encoder/layer_2/attention/self/query/bias:0
19 bert/encoder/layer_2/attention/self/key/kernel:0
20 bert/encoder/l

164 bert/encoder/layer_7/output/dense/bias:0
165 bert/encoder/layer_8/output/dense/kernel:0
166 bert/encoder/layer_8/output/dense/bias:0
167 bert/encoder/layer_9/output/dense/kernel:0
168 bert/encoder/layer_9/output/dense/bias:0
169 bert/encoder/layer_10/output/dense/kernel:0
170 bert/encoder/layer_10/output/dense/bias:0
171 bert/encoder/layer_11/output/dense/kernel:0
172 bert/encoder/layer_11/output/dense/bias:0
173 bert/encoder/layer_0/output/LayerNorm/gamma:0
174 bert/encoder/layer_0/output/LayerNorm/beta:0
175 bert/encoder/layer_1/output/LayerNorm/gamma:0
176 bert/encoder/layer_1/output/LayerNorm/beta:0
177 bert/encoder/layer_2/output/LayerNorm/gamma:0
178 bert/encoder/layer_2/output/LayerNorm/beta:0
179 bert/encoder/layer_3/output/LayerNorm/gamma:0
180 bert/encoder/layer_3/output/LayerNorm/beta:0
181 bert/encoder/layer_4/output/LayerNorm/gamma:0
182 bert/encoder/layer_4/output/LayerNorm/beta:0
183 bert/encoder/layer_5/output/LayerNorm/gamma:0
184 bert/encoder/layer_5/output/LayerN

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (BertModel)                (None, 768)          177853440   input_ids[0][0]                  
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
______________________________________________________________________________________________

In [16]:
model

In [19]:
# convert.segment_ids

In [20]:
import numpy as np
np.array(convert.input_ids).reshape(1,128)

array([[  101, 76294, 11744, 25922,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [17]:
a = model.predict([np.array(convert.input_ids).reshape(1,128), np.array(convert.input_mask).reshape(1,128), np.array(convert.segment_ids).reshape(1,128)])

In [18]:
a

array([[0.31052852, 0.6488204 ]], dtype=float32)

In [21]:
a

array([[0.20167862, 0.30923742]], dtype=float32)

In [31]:
a.shape

(1, 128, 768)

In [21]:
actual_names = ['bert/embeddings/word_embeddings/embeddings:0',
 'bert/embeddings/token_type_embeddings/embeddings:0',
 'bert/embeddings/position_embeddings/embeddings:0',
 'bert/embeddings/LayerNorm/gamma:0',
 'bert/embeddings/LayerNorm/beta:0',
 'bert/encoder/layer_0/attention/self/query/kernel:0',
 'bert/encoder/layer_0/attention/self/query/bias:0',
 'bert/encoder/layer_0/attention/self/key/kernel:0',
 'bert/encoder/layer_0/attention/self/key/bias:0',
 'bert/encoder/layer_0/attention/self/value/kernel:0',
 'bert/encoder/layer_0/attention/self/value/bias:0',
 'bert/encoder/layer_0/attention/output/dense/kernel:0',
 'bert/encoder/layer_0/attention/output/dense/bias:0',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_0/intermediate/kernel:0',
 'bert/encoder/layer_0/intermediate/bias:0',
 'bert/encoder/layer_0/output/dense/kernel:0',
 'bert/encoder/layer_0/output/dense/bias:0',
 'bert/encoder/layer_0/output/LayerNorm/gamma:0',
 'bert/encoder/layer_0/output/LayerNorm/beta:0',
 'bert/encoder/layer_1/attention/self/query/kernel:0',
 'bert/encoder/layer_1/attention/self/query/bias:0',
 'bert/encoder/layer_1/attention/self/key/kernel:0',
 'bert/encoder/layer_1/attention/self/key/bias:0',
 'bert/encoder/layer_1/attention/self/value/kernel:0',
 'bert/encoder/layer_1/attention/self/value/bias:0',
 'bert/encoder/layer_1/attention/output/dense/kernel:0',
 'bert/encoder/layer_1/attention/output/dense/bias:0',
 'bert/encoder/layer_1/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_1/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_1/intermediate/kernel:0',
 'bert/encoder/layer_1/intermediate/bias:0',
 'bert/encoder/layer_1/output/dense/kernel:0',
 'bert/encoder/layer_1/output/dense/bias:0',
 'bert/encoder/layer_1/output/LayerNorm/gamma:0',
 'bert/encoder/layer_1/output/LayerNorm/beta:0',
 'bert/encoder/layer_2/attention/self/query/kernel:0',
 'bert/encoder/layer_2/attention/self/query/bias:0',
 'bert/encoder/layer_2/attention/self/key/kernel:0',
 'bert/encoder/layer_2/attention/self/key/bias:0',
 'bert/encoder/layer_2/attention/self/value/kernel:0',
 'bert/encoder/layer_2/attention/self/value/bias:0',
 'bert/encoder/layer_2/attention/output/dense/kernel:0',
 'bert/encoder/layer_2/attention/output/dense/bias:0',
 'bert/encoder/layer_2/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_2/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_2/intermediate/kernel:0',
 'bert/encoder/layer_2/intermediate/bias:0',
 'bert/encoder/layer_2/output/dense/kernel:0',
 'bert/encoder/layer_2/output/dense/bias:0',
 'bert/encoder/layer_2/output/LayerNorm/gamma:0',
 'bert/encoder/layer_2/output/LayerNorm/beta:0',
 'bert/encoder/layer_3/attention/self/query/kernel:0',
 'bert/encoder/layer_3/attention/self/query/bias:0',
 'bert/encoder/layer_3/attention/self/key/kernel:0',
 'bert/encoder/layer_3/attention/self/key/bias:0',
 'bert/encoder/layer_3/attention/self/value/kernel:0',
 'bert/encoder/layer_3/attention/self/value/bias:0',
 'bert/encoder/layer_3/attention/output/dense/kernel:0',
 'bert/encoder/layer_3/attention/output/dense/bias:0',
 'bert/encoder/layer_3/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_3/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_3/intermediate/kernel:0',
 'bert/encoder/layer_3/intermediate/bias:0',
 'bert/encoder/layer_3/output/dense/kernel:0',
 'bert/encoder/layer_3/output/dense/bias:0',
 'bert/encoder/layer_3/output/LayerNorm/gamma:0',
 'bert/encoder/layer_3/output/LayerNorm/beta:0',
 'bert/encoder/layer_4/attention/self/query/kernel:0',
 'bert/encoder/layer_4/attention/self/query/bias:0',
 'bert/encoder/layer_4/attention/self/key/kernel:0',
 'bert/encoder/layer_4/attention/self/key/bias:0',
 'bert/encoder/layer_4/attention/self/value/kernel:0',
 'bert/encoder/layer_4/attention/self/value/bias:0',
 'bert/encoder/layer_4/attention/output/dense/kernel:0',
 'bert/encoder/layer_4/attention/output/dense/bias:0',
 'bert/encoder/layer_4/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_4/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_4/intermediate/kernel:0',
 'bert/encoder/layer_4/intermediate/bias:0',
 'bert/encoder/layer_4/output/dense/kernel:0',
 'bert/encoder/layer_4/output/dense/bias:0',
 'bert/encoder/layer_4/output/LayerNorm/gamma:0',
 'bert/encoder/layer_4/output/LayerNorm/beta:0',
 'bert/encoder/layer_5/attention/self/query/kernel:0',
 'bert/encoder/layer_5/attention/self/query/bias:0',
 'bert/encoder/layer_5/attention/self/key/kernel:0',
 'bert/encoder/layer_5/attention/self/key/bias:0',
 'bert/encoder/layer_5/attention/self/value/kernel:0',
 'bert/encoder/layer_5/attention/self/value/bias:0',
 'bert/encoder/layer_5/attention/output/dense/kernel:0',
 'bert/encoder/layer_5/attention/output/dense/bias:0',
 'bert/encoder/layer_5/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_5/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_5/intermediate/kernel:0',
 'bert/encoder/layer_5/intermediate/bias:0',
 'bert/encoder/layer_5/output/dense/kernel:0',
 'bert/encoder/layer_5/output/dense/bias:0',
 'bert/encoder/layer_5/output/LayerNorm/gamma:0',
 'bert/encoder/layer_5/output/LayerNorm/beta:0',
 'bert/encoder/layer_6/attention/self/query/kernel:0',
 'bert/encoder/layer_6/attention/self/query/bias:0',
 'bert/encoder/layer_6/attention/self/key/kernel:0',
 'bert/encoder/layer_6/attention/self/key/bias:0',
 'bert/encoder/layer_6/attention/self/value/kernel:0',
 'bert/encoder/layer_6/attention/self/value/bias:0',
 'bert/encoder/layer_6/attention/output/dense/kernel:0',
 'bert/encoder/layer_6/attention/output/dense/bias:0',
 'bert/encoder/layer_6/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_6/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_6/intermediate/kernel:0',
 'bert/encoder/layer_6/intermediate/bias:0',
 'bert/encoder/layer_6/output/dense/kernel:0',
 'bert/encoder/layer_6/output/dense/bias:0',
 'bert/encoder/layer_6/output/LayerNorm/gamma:0',
 'bert/encoder/layer_6/output/LayerNorm/beta:0',
 'bert/encoder/layer_7/attention/self/query/kernel:0',
 'bert/encoder/layer_7/attention/self/query/bias:0',
 'bert/encoder/layer_7/attention/self/key/kernel:0',
 'bert/encoder/layer_7/attention/self/key/bias:0',
 'bert/encoder/layer_7/attention/self/value/kernel:0',
 'bert/encoder/layer_7/attention/self/value/bias:0',
 'bert/encoder/layer_7/attention/output/dense/kernel:0',
 'bert/encoder/layer_7/attention/output/dense/bias:0',
 'bert/encoder/layer_7/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_7/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_7/intermediate/kernel:0',
 'bert/encoder/layer_7/intermediate/bias:0',
 'bert/encoder/layer_7/output/dense/kernel:0',
 'bert/encoder/layer_7/output/dense/bias:0',
 'bert/encoder/layer_7/output/LayerNorm/gamma:0',
 'bert/encoder/layer_7/output/LayerNorm/beta:0',
 'bert/encoder/layer_8/attention/self/query/kernel:0',
 'bert/encoder/layer_8/attention/self/query/bias:0',
 'bert/encoder/layer_8/attention/self/key/kernel:0',
 'bert/encoder/layer_8/attention/self/key/bias:0',
 'bert/encoder/layer_8/attention/self/value/kernel:0',
 'bert/encoder/layer_8/attention/self/value/bias:0',
 'bert/encoder/layer_8/attention/output/dense/kernel:0',
 'bert/encoder/layer_8/attention/output/dense/bias:0',
 'bert/encoder/layer_8/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_8/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_8/intermediate/kernel:0',
 'bert/encoder/layer_8/intermediate/bias:0',
 'bert/encoder/layer_8/output/dense/kernel:0',
 'bert/encoder/layer_8/output/dense/bias:0',
 'bert/encoder/layer_8/output/LayerNorm/gamma:0',
 'bert/encoder/layer_8/output/LayerNorm/beta:0',
 'bert/encoder/layer_9/attention/self/query/kernel:0',
 'bert/encoder/layer_9/attention/self/query/bias:0',
 'bert/encoder/layer_9/attention/self/key/kernel:0',
 'bert/encoder/layer_9/attention/self/key/bias:0',
 'bert/encoder/layer_9/attention/self/value/kernel:0',
 'bert/encoder/layer_9/attention/self/value/bias:0',
 'bert/encoder/layer_9/attention/output/dense/kernel:0',
 'bert/encoder/layer_9/attention/output/dense/bias:0',
 'bert/encoder/layer_9/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_9/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_9/intermediate/kernel:0',
 'bert/encoder/layer_9/intermediate/bias:0',
 'bert/encoder/layer_9/output/dense/kernel:0',
 'bert/encoder/layer_9/output/dense/bias:0',
 'bert/encoder/layer_9/output/LayerNorm/gamma:0',
 'bert/encoder/layer_9/output/LayerNorm/beta:0',
 'bert/encoder/layer_10/attention/self/query/kernel:0',
 'bert/encoder/layer_10/attention/self/query/bias:0',
 'bert/encoder/layer_10/attention/self/key/kernel:0',
 'bert/encoder/layer_10/attention/self/key/bias:0',
 'bert/encoder/layer_10/attention/self/value/kernel:0',
 'bert/encoder/layer_10/attention/self/value/bias:0',
 'bert/encoder/layer_10/attention/output/dense/kernel:0',
 'bert/encoder/layer_10/attention/output/dense/bias:0',
 'bert/encoder/layer_10/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_10/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_10/intermediate/kernel:0',
 'bert/encoder/layer_10/intermediate/bias:0',
 'bert/encoder/layer_10/output/dense/kernel:0',
 'bert/encoder/layer_10/output/dense/bias:0',
 'bert/encoder/layer_10/output/LayerNorm/gamma:0',
 'bert/encoder/layer_10/output/LayerNorm/beta:0',
 'bert/encoder/layer_11/attention/self/query/kernel:0',
 'bert/encoder/layer_11/attention/self/query/bias:0',
 'bert/encoder/layer_11/attention/self/key/kernel:0',
 'bert/encoder/layer_11/attention/self/key/bias:0',
 'bert/encoder/layer_11/attention/self/value/kernel:0',
 'bert/encoder/layer_11/attention/self/value/bias:0',
 'bert/encoder/layer_11/attention/output/dense/kernel:0',
 'bert/encoder/layer_11/attention/output/dense/bias:0',
 'bert/encoder/layer_11/attention/output/LayerNorm/gamma:0',
 'bert/encoder/layer_11/attention/output/LayerNorm/beta:0',
 'bert/encoder/layer_11/intermediate/kernel:0',
 'bert/encoder/layer_11/intermediate/bias:0',
 'bert/encoder/layer_11/output/dense/kernel:0',
 'bert/encoder/layer_11/output/dense/bias:0',
 'bert/encoder/layer_11/output/LayerNorm/gamma:0',
 'bert/encoder/layer_11/output/LayerNorm/beta:0']


In [33]:
# for idx, (actual, curr) in enumerate(zip(actual_names, bert.weights)):
# #     if actual != curr.name:
#     if not map_to_stock_variable_name(actual) == map_to_stock_variable_name_v2(curr.name):
#         print (idx, actual, map_to_stock_variable_name(actual) , map_to_stock_variable_name_v2(curr.name))

In [36]:
# for idx, each in enumerate(bert.weights):
#     print(idx, each.name, map_to_stock_variable_name(each.name))
    
    